# LangChain Agents Demo 🚀

### Author: **Cristiano Casadei** - R&D Maggioli Group

In [ ]:
# download resources

!wget https://cristianofrstorage.blob.core.windows.net/rnddays2023/RnD.tar.gz && tar xvzf RnD.tar.gz

In [ ]:
# I need to install some libriaries...

!pip install -r requirements.txt -q

In [ ]:
# load environment variables with some secrets (will be disabled in a week...)

from dotenv import load_dotenv

load_dotenv("env")

In [ ]:
# initialize the LLM to use
# I use GPT-3.5-Turbo that is a Chat model

from langchain.llms import OpenAIChat

llm = OpenAIChat(deployment_id="mg-gpt35-turbo", temperature=0)

In [ ]:
# initialize the tools and the agent

from langchain.agents import initialize_agent, AgentType, Tool
from langchain.utilities import GoogleSearchAPIWrapper, PythonREPL
from langchain.tools import PythonAstREPLTool

# this tool collect information on Google Search
search = GoogleSearchAPIWrapper()

# this tool can run a Python script
python_repl = PythonAstREPLTool()

# define a list of tools made available to the library
# the description is useful for agent to decide which tool to use
toolkit = [
    Tool(
        name="google-search",
        func=search.run,
        description="useful for when you need to search Google to answer questions about current events"
    ),
    Tool(
        name="python-shell",
        func=python_repl.run,
        description="A Python shell. Use this to execute Python commands. " \
                    "Input should be a valid Python multiline command. " \
                    "If you want to see the output of a value, you should print it out with `print(...)`."
    )
]

# initialize the agent 
agent = initialize_agent(
    toolkit,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True  
)

### <span style="color:yellow">⚠⚠</span><span style="color:red"> NOTE: logs with the agent's "mental flows" will be listed under the UI </span><span style="color:yellow">⚠⚠</span>

### Some example prompts

```
genera un articolo di giornale sui terremoti dei Campi Flegrei degli ultimi 2 mesi.
ricerca le informazioni che ti servono in internet.
spiega qual'è stata fin'ora l'evoluzione delle scosse, qual'è il sentimento dei cittadini e come si stanno muovendo le istituzioni per far fronte ad eventuali stati di crisi che si venissero a creare a fronte di un terremoto devastante.
scrivi il risultato in un file "output/articolo.txt"
```
___
```
scrivi un file "output/prova2.csv".
come intestazione usa "Nome,Cognome,Età,Residenza".
riempi il file con l'intestazione seguita da 100 righe con nomi e cognomi italiani (casuali), età casuale compresa tra 18 e 60 anni, nomi di città italiane (casuali) come residenza, separati da virgola.
```
___
```
leggi il file "dati/dati_esempio.csv" ed elenca le persone che hanno uno stipendio <= 1.400€
```
___
```
leggi il file "dati/dati_esempio.csv" e riporta le informazioni in una tabella di nome "esempio" di un database sqlite salvato su file "output/database.db"
```
___

In [ ]:
import gradio as gr

# I create a simple UI with a text input and text output
demo = gr.Interface(
    fn=lambda command: agent(command)["output"],
    inputs="text",
    outputs="text"
)

demo.launch(share=True)

In [ ]:
%load_ext sql
%sql sqlite:///output/database.db

In [ ]:
%%sql

select * from esempio